### [How to chain runnables](https://python.langchain.com/docs/how_to/sequence/)

In [1]:
import getpass
import os

if "LANGCHAIN_API_KEY" not in os.environ:
    os.environ["LANGCHAIN_TRACING_V2"] = "true"
    os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

In [2]:
if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass()

#### A. Model

In [3]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini")

In [4]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("Cuentame un chiste de {topic}")

chain = prompt | model | StrOutputParser()

# chain.invoke({"topic": "programacion"})
chunks = []
for chunk in chain.stream({"topic": "programacion"}):
    chunks.append(chunk)
    print(chunk, end='|', flush=True)

|¡|Claro|!| Aquí| tienes| uno|:

|¿Por| qué| los| program|adores| pref|ieren| el| café| oscuro|?

|¡|Porque| la| luz| atra|e| a| los| bugs|!| ☕|🐛|

|Espero| que| te| haya| sac|ado| una| sonrisa|.| Si| quieres| otro|,| ¡|d|ím|elo|!||

#### B. Coercion

In [5]:
from langchain_core.output_parsers import StrOutputParser

analysis_prompt = ChatPromptTemplate.from_template("Analiza el chiste: {joke}")

composed_chain = {"joke": chain} | analysis_prompt | model | StrOutputParser()

# composed_chain.invoke({"topic": "programacion"})
chunks = []
for chunk in composed_chain.stream({"topic": "programacion"}):
    chunks.append(chunk)
    print(chunk, end='|', flush=True)

|Este| ch|iste| juega| con| un| par| de| conceptos| que| son| bien| conocidos| en| el| mundo| de| la| programación| y| la| informática|.

|1|.| **|C|afé| oscuro|**|:| La| primera| parte| del| ch|iste| establece| una| prefer|encia| por| el| café| oscuro|,| que| es| una| elección| común| entre| muchas| personas|,| pero| aquí| se| utiliza| como| un| recurso| para| conectar| con| el| contexto| de| los| program|adores|.

|2|.| **|L|uz| y| bugs|**|:| El| núcleo| del| ch|iste| rad|ica| en| el| juego| de| palabras| con| "|bugs|".| En| el| ámbito| de| la| programación|,| "|bugs|"| se| refiere| a| errores| o| fall|os| en| el| código|.| Sin| embargo|,| el| término| "|bug|"| también| hace| referencia| a| insect|os| que|,| en| la| vida| real|,| son| atra|ídos| por| la| luz|.| Este| doble| sentido| crea| una| conexión| humor|ística| entre| la| prefer|encia| del| program|ador| por| el| café| oscuro| y| la| idea| de| que| la| luz| (|que| atra|e| a| los| insect|os|)| también| atra|e| a| los| errores| e

#### C. Test

In [37]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

# Paso 1: Crear el primer chain para generar un chiste
joke_prompt = ChatPromptTemplate.from_template("Cuentame un chiste con un máximo de 10 palabras, acerca de {topic}")
joke_chain = joke_prompt | model | StrOutputParser()

# Paso 2: Crear el segundo chain para analizar el chiste
analysis_prompt = ChatPromptTemplate.from_template("Analiza el chiste: {joke}, Inicia con, 'ANALIZANDO EL CHISTE' y finaliza con 'EN RESUMEN: '")
composed_chain = {"joke": joke_chain} | analysis_prompt | model | StrOutputParser()

# Paso 3: Realizar el streaming del composed_chain
chunks = []
for chunk in composed_chain.stream({"topic": "programacion"}):
    chunks.append(chunk)
    print(chunk, end='|', flush=True)

|AN|AL|IZ|ANDO| EL| CH|IST|E|:| 

|Este| ch|iste| juega| con| un| juego| de| palabras| que| combina| dos| signific|ados|.| Por| un| lado|,| está| la| referencia| al| café|,| una| bebida| popular| entre| los| program|adores| por| su| capacidad| para| mantener|los| despi|ertos| y| concentr|ados| durante| largas| horas| de| trabajo|.| Por| otro| lado|,| "|Java|"| es| el| nombre| de| un| popular| lenguaje| de| programación| utilizado| en| el| desarrollo| de| software|.| La| conexión| entre| el| café| y| "|Java|"| se| establece| a| través| de| la| hom|of|on|ía|,| ya| que| "|java|"| es| también| un| término| colo|quial| para| refer|irse| al| café|,| especialmente| en| algunos| context|os|.

|El| humor| se| deriva| del| doble| sentido|:| el| oy|ente| puede| interpretar| la| frase| de| manera| literal|,| pensando| en| cómo| el| café| ayuda| a| los| program|adores| a| mantenerse| alert|as|,| o| en| un| sentido| más| técnico|,| donde| "|Java|"| se| refiere| al| lenguaje| de| programación|.| Este

In [38]:
events = []
async for event in composed_chain.astream_events({"topic": "programacion"}, version="v2"):
    events.append(event)

event_types = {event["event"] for event in events}
print("Unique event types:", event_types)

Unique event types: {'on_prompt_start', 'on_chain_stream', 'on_parser_end', 'on_prompt_end', 'on_chain_start', 'on_parser_start', 'on_chat_model_end', 'on_chat_model_stream', 'on_parser_stream', 'on_chain_end', 'on_chat_model_start'}


In [40]:
async for event in composed_chain.astream_events("programacion", version="v2"):
    if event["event"] == "on_chat_model_start":
        print("Stream started...", flush=True)
    elif event["event"] == "on_chat_model_stream":
        print(event["data"]["chunk"].content, end='|', flush=True)
    elif event["event"] == "on_chat_model_end":
        print("\nStream ended...", flush=True)

Stream started...
|¿Por| qué| los| program|adores| conf|unden| Halloween| y| Navidad|?| ¡|Porque| Oct| |31| es| Dec| |25|!||
Stream ended...
Stream started...
|AN|AL|IZ|ANDO| EL| CH|IST|E|:

|Este| ch|iste| juega| con| el| conocimiento| de| sistemas| num|ér|icos| y| las| convenc|iones| que| se| utilizan| en| programación|.| En| él|,| se| menc|iona| que| los| program|adores| conf|unden| Halloween| (|31| de| octubre|)| y| Navidad| (|25| de| diciembre|)| debido| a| una| relación| num|érica| entre| las| fechas|.| La| clave| del| ch|iste| rad|ica| en| la| interpretación| de| los| números| en| diferentes| bases|:

|-| "|Oct| |31|"| se| refiere| a| "|31| en| base| oct|al|"| (|base| |8|),| que| en| el| sistema| decimal| (|base| |10|)| equiv|ale| a| |25|.
|-| "|Dec| |25|"| se| refiere| a| "|25| en| base| decimal|"| (|base| |10|).

|La| conf|usión| surge| porque| "|Oct|"| y| "|Dec|"| también| se| pueden| interpretar| como| abre|vi|aturas| de| "|oct|al|"| y| "|decimal|",| respectivamente|.| Por| 

In [26]:
events = [
    event
    async for event in composed_chain.astream_events(
        {"topic": "programacion"},
        version="v2",
    )
]

In [27]:
events

[{'event': 'on_chain_start',
  'data': {'input': {'topic': 'programacion'}},
  'name': 'RunnableSequence',
  'tags': [],
  'run_id': 'e99215c1-d6ba-4b9f-bc24-c0ea6bf281dc',
  'metadata': {},
  'parent_ids': []},
 {'event': 'on_chain_start',
  'data': {},
  'name': 'RunnableParallel<joke>',
  'tags': ['seq:step:1'],
  'run_id': 'df56a401-b888-45e3-9e52-321d8fc28b9a',
  'metadata': {},
  'parent_ids': ['e99215c1-d6ba-4b9f-bc24-c0ea6bf281dc']},
 {'event': 'on_chain_start',
  'data': {},
  'name': 'RunnableSequence',
  'tags': ['map:key:joke'],
  'run_id': 'f578e6c4-215c-4120-a2dc-86e48585f702',
  'metadata': {},
  'parent_ids': ['e99215c1-d6ba-4b9f-bc24-c0ea6bf281dc',
   'df56a401-b888-45e3-9e52-321d8fc28b9a']},
 {'event': 'on_prompt_start',
  'data': {'input': {'topic': 'programacion'}},
  'name': 'ChatPromptTemplate',
  'tags': ['seq:step:1'],
  'run_id': '2d5494f1-d3e7-4cd8-879a-99815ff5ebe0',
  'metadata': {},
  'parent_ids': ['e99215c1-d6ba-4b9f-bc24-c0ea6bf281dc',
   'df56a401-b888-

In [11]:
num_events = 0

async for event in composed_chain.astream_events({"topic": "programacion"}, version="v2"):
    kind = event["event"]
    
    if kind == "on_chat_model_stream":
        chunk = event["data"]["chunk"].content
        print(f"Chat model chunk: {repr(chunk)}", flush=True)

    elif kind == "on_parser_stream":
        chunk = event["data"]["chunk"]
        print(f"Parser chunk: {chunk}", flush=True)

    elif kind == "on_chain_stream":
        chunk = event["data"].get("chunk", None)
        if isinstance(chunk, dict):
            for key, value in chunk.items():
                print(f"Chain {key} chunk: {repr(value)}", flush=True)
        else:
            print(f"Chain chunk: {repr(chunk)}", flush=True)

    num_events += 1

    if num_events > 30:  # Truncate output if too many events
        print("...", flush=True)
        break

Chat model chunk: ''
Parser chunk: 
Chain chunk: ''
Chain joke chunk: ''
Chat model chunk: '¡'
Parser chunk: ¡
Chain chunk: '¡'
Chain joke chunk: '¡'
Chat model chunk: 'Claro'
Parser chunk: Claro
Chain chunk: 'Claro'
Chain joke chunk: 'Claro'
Chat model chunk: '!'
Parser chunk: !
Chain chunk: '!'
Chain joke chunk: '!'
Chat model chunk: ' Aquí'
Parser chunk:  Aquí
Chain chunk: ' Aquí'
Chain joke chunk: ' Aquí'
Chat model chunk: ' tienes'
Parser chunk:  tienes
Chain chunk: ' tienes'
Chain joke chunk: ' tienes'
...


#### D. Metodo pipe

In [12]:
from langchain_core.runnables import RunnableParallel

composed_chain_with_pipe = (
    RunnableParallel({"joke": chain})
    .pipe(analysis_prompt)
    .pipe(model)
    .pipe(StrOutputParser())
)

composed_chain_with_pipe.invoke({"topic": "battlestar galactica"})

'El chiste se basa en la premisa de la serie "Battlestar Galactica", donde los cylons son seres artificiales diseñados para cazar y exterminar a la humanidad. La broma juega con la idea de que, debido a su naturaleza y habilidades, los cylons serían extremadamente efectivos en un juego como las escondidas, donde el objetivo es encontrar a los demás jugadores.\n\nEl humor proviene de la exageración de la capacidad de los cylons para localizar a los humanos, convirtiendo una situación lúdica en una referencia a su papel como antagonistas en la serie. La incongruencia entre la seriedad del contexto (la caza de humanos) y la ligereza del juego (escondidas) crea un efecto cómico.\n\nEN RESUMEN: El chiste utiliza la conexión entre los cylons y su habilidad para encontrar humanos para recontextualizar un juego infantil, generando humor a través de la ironía y la exageración.'

In [13]:
composed_chain_with_pipe = RunnableParallel({"joke": chain}).pipe(
    analysis_prompt, model, StrOutputParser()
)

chunks = []
for chunk in composed_chain_with_pipe.stream({"topic": "programacion"}):
    chunks.append(chunk)
    print(chunk, end='|', flush=True)

|Este| ch|iste| juega| con| el| doble| sentido| de| la| palabra| "|light|".| Por| un| lado|,| se| refiere| a| la| prefer|encia| de| los| program|adores| por| el| café| oscuro|,| que| es| más| fuerte| y| concentr|ado|,| lo| que| podría| asoci|arse| con| la| necesidad| de| energía| para| realizar| su| trabajo|.| Por| otro| lado|,| "|light|"| también| puede| refer|irse| a| un| código| que| es| poco| d|enso| o| fácil|,| lo| cual| puede| interpret|arse| como| un| código| que| no| es| lo| suficientemente| robust|o| o| efectivo|.| 

|La| b|roma| su|gi|ere| que| los| program|adores| pref|ieren| un| café| que| les| mant|enga| alerta| y| enf|ocados|,| al| igual| que| buscan| que| su| código| sea| sólido| y| no| "|lig|ero|"| o| superficial|.| El| juego| de| palabras| añade| un| toque| de| humor| que| puede| reson|ar| especialmente| con| quienes| están| familiar|izados| con| la| programación| y| su| cultura|.

|EN| RES|UM|EN|:| El| ch|iste| utiliza| un| juego| de| palabras| entre| "|light|"| como|